In [1]:
import keras
from keras import models, layers, applications, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np

from scipy.ndimage import imread
from scipy.misc import imresize

# from keras_models.imagenet_utils import preprocess_input
# from keras_models.inception_v3 import InceptionV3

from nbutil import imshow_multi
from PIL import Image

import random
import os

Using TensorFlow backend.


# Load the data

Load existing training and test data, and resize to 64x64

In [2]:
data_dir = '../data/proj6/data/15SceneData'
categories = [c for c in os.listdir(os.path.join(data_dir, 'train')) if c[0] != '.']

SIZE = 139

def load_dataset(name):
    images = []
    labels = []
    for cat in categories:
        for filename in os.listdir(os.path.join(data_dir, name, cat)):
            if filename.endswith('.jpg'): 
                path = os.path.join(data_dir, name, cat, filename)
                image = imread(path)
                h,w = image.shape[:2]
                # top-left crop
                if h > w:
                    image = image[:w, :]
                else:
                    image = image[:, :h]
                image = imresize(image, size=[SIZE, SIZE], mode='L')
                images.append(image)
                labels.append(categories.index(cat))
    images = np.expand_dims(np.stack(images) / 255.0, -1)
    labels = np.expand_dims(np.stack(labels).astype(int), -1)
    return images, labels

trainx, trainy = load_dataset('train')
testx, testy = load_dataset('test')


In [57]:
y_dim = max(trainy)[0] + 1
y_dim

15

# Do some jittering and centering

Use Keras's `ImageDataGenerator` to apply image jitter. Compute the mean image using the training data.

In [3]:
image_gen = ImageDataGenerator(rotation_range=5, 
                               zoom_range=0.2, 
                               width_shift_range=0.1, 
                               height_shift_range=0.1, 
                               horizontal_flip=True,
                               channel_shift_range=0.2)
# samplewise centering is broken in keras (https://github.com/fchollet/keras/issues/4047)
# so do it manually:
# mean_img = np.mean(trainx, axis=0)


# Subtract the mean image and convert all to RGB

(since InceptionV3 takes 3-channel inputs)

In [58]:
def inceptionv3_preprocess(x):
    # recenter around 0:
    x = (x - 0.5) * 2
    # inception wants 3 channels, not 1:
    x = np.repeat(x, 3, axis=-1)
    return x

trainx_rgb = inceptionv3_preprocess(trainx - mean_img)
testx_rgb = inceptionv3_preprocess(testx - mean_img)

In [59]:
trainx_rgb.shape

(1500, 128, 128, 3)

# Load a pre-trained network

Pretrained `VGG-F` networks aren't available for Keras, so use `InceptionV3` instead.

In [14]:
input = layers.Input(shape=(SIZE, SIZE, 3))
model = applications.InceptionV3(weights='imagenet', include_top=False, input_tensor=input)
# model = InceptionV3(weights='imagenet', include_top=False)

In [23]:
# model.summary()

# Create a new model

In [78]:
output = model.get_layer('mixed5').get_output_at(0)
print(output.shape)
output = layers.Conv2D(256, (1,1), activation='elu')(output)
print(output.shape)
output = layers.Dropout(0.5)(output)
output = layers.AveragePooling2D((6,6))(output)
print(output.shape)
output = layers.Flatten()(output)
print(output.shape)
output = layers.Dense(y_dim, activation='softmax')(output)
model2 = models.Model(inputs=[input], outputs=[output])

(?, 6, 6, 768)
(?, 6, 6, 256)
(?, 1, 1, 256)
(?, ?)


In [79]:
y = model2.predict(trainx_rgb[:5])
y.shape

(5, 15)

In [83]:
model2.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

In [85]:

batch_size = 8

for i in range(40):
    steps = len(trainx)/batch_size
    # steps = 1
    model2.optimizer.lr.assign((1e-4) / 2)
    o = model2.fit_generator(image_gen.flow(trainx_rgb, trainy, batch_size=batch_size),
                       steps_per_epoch=steps,
                       verbose=1,
                       workers=2,
                       epochs=1)
    if i % 1 == 0:
        score = model2.evaluate(testx_rgb, testy, verbose=0)
        print('epoch: {}, loss: {}, accuracy: {}'.format(i+1, score[0], score[1]))

Epoch 1/1
188/187 [==============================] - 25s - loss: 0.5199 - acc: 0.8504    
epoch: 1, loss: 0.5351562928314784, accuracy: 0.8244556114102528
Epoch 1/1
188/187 [==============================] - 25s - loss: 0.4767 - acc: 0.8404    
epoch: 2, loss: 0.5103619897784899, accuracy: 0.8328308207904873
Epoch 1/1
188/187 [==============================] - 25s - loss: 0.4569 - acc: 0.8564    
epoch: 3, loss: 0.5124339959729257, accuracy: 0.8294807370583616
Epoch 1/1
188/187 [==============================] - 25s - loss: 0.4201 - acc: 0.8710    
epoch: 4, loss: 0.5103467443879924, accuracy: 0.8355108877921623
Epoch 1/1
188/187 [==============================] - 25s - loss: 0.3663 - acc: 0.8956    
epoch: 5, loss: 0.5447332611834783, accuracy: 0.8288107202879748
Epoch 1/1
188/187 [==============================] - 26s - loss: 0.3712 - acc: 0.8670    
epoch: 6, loss: 0.5340726169010303, accuracy: 0.8318257956648591
Epoch 1/1
  6/187 [..............................] - ETA: 192s - loss:

KeyboardInterrupt: 